In [5]:
import pandas as pd

# Load the datasets
match_df = pd.read_csv('./match_level_scorecard.csv')
bowler_df = pd.read_csv('./bowler_level_scorecard.csv')
batsman_df = pd.read_csv('./batsman_level_scorecard.csv')

# Convert match date to datetime for sorting
match_df['match_dt'] = pd.to_datetime(match_df['match_dt'])
batsman_df['match_dt'] = pd.to_datetime(batsman_df['match_dt'])
bowler_df['match_dt'] = pd.to_datetime(bowler_df['match_dt'])

# Function to convert string roster IDs to a list of integers
def convert_roster_ids(roster_ids):
    return [int(float(x)) for x in roster_ids.split(':')]

# Calculate team1's average runs in last 5 games
def calculate_avg_runs_last_5(team_id, batsman_df):
    team_batsman_df = batsman_df[batsman_df['batsman_id'].isin([team_id])]
    last_5_games = team_batsman_df.nlargest(5, 'match_dt')
    return last_5_games['runs'].mean()

match_df['team1_avg_runs_last_5'] = match_df['team1_id'].apply(lambda team1_id: calculate_avg_runs_last_5(team1_id, batsman_df))
match_df['team2_avg_runs_last_5'] = match_df['team2_id'].apply(lambda team2_id: calculate_avg_runs_last_5(team2_id, batsman_df))

# Calculate average economy rate of bowlers in last 5 games
def calculate_avg_economy_last_5(team_id, bowler_df):
    team_bowler_df = bowler_df[bowler_df['bowler_id'].isin([team_id])]
    last_5_games = team_bowler_df.nlargest(5, 'match_dt')
    return last_5_games['economy'].mean()

match_df['team1_bowler_economy_last_5'] = match_df['team1_id'].apply(lambda team1_id: calculate_avg_economy_last_5(team1_id, bowler_df))
match_df['team2_bowler_economy_last_5'] = match_df['team2_id'].apply(lambda team2_id: calculate_avg_economy_last_5(team2_id, bowler_df))

# Calculate total wickets taken by bowlers in last 5 games
def calculate_wickets_last_5(team_id, bowler_df):
    team_bowler_df = bowler_df[bowler_df['bowler_id'].isin([team_id])]
    last_5_games = team_bowler_df.nlargest(5, 'match_dt')
    return last_5_games['wicket_count'].sum()

match_df['team1_wickets_last_5'] = match_df['team1_id'].apply(lambda team1_id: calculate_wickets_last_5(team1_id, bowler_df))
match_df['team2_wickets_last_5'] = match_df['team2_id'].apply(lambda team2_id: calculate_wickets_last_5(team2_id, bowler_df))

# Calculate total boundaries scored by team in last 5 games
def calculate_boundaries_last_5(team_id, batsman_df):
    team_batsman_df = batsman_df[batsman_df['batsman_id'].isin([team_id])]
    last_5_games = team_batsman_df.nlargest(5, 'match_dt')
    return last_5_games['Fours'].sum() + last_5_games['Sixes'].sum()

match_df['team1_boundaries_last_5'] = match_df['team1_id'].apply(lambda team1_id: calculate_boundaries_last_5(team1_id, batsman_df))
match_df['team2_boundaries_last_5'] = match_df['team2_id'].apply(lambda team2_id: calculate_boundaries_last_5(team2_id, batsman_df))

# Create toss_win_impact feature
match_df['toss_win_impact'] = (match_df['toss winner'] == match_df['winner']).astype(int)


# Calculate team experience based on number of matches played by roster players
def calculate_team_experience(roster_ids, batsman_df):
    roster = convert_roster_ids(roster_ids)
    experience = batsman_df[batsman_df['batsman_id'].isin(roster)]['match id'].nunique()
    return experience / len(roster)

match_df['team1_experience'] = match_df['team1_roster_ids'].apply(lambda roster_ids: calculate_team_experience(roster_ids, batsman_df))
match_df['team2_experience'] = match_df['team2_roster_ids'].apply(lambda roster_ids: calculate_team_experience(roster_ids, batsman_df))

# Create a DataFrame with match_id and the newly created additional features
additional_features_df = match_df[['match id', 
                                   'team1_avg_runs_last_5', 
                                   'team2_avg_runs_last_5', 
                                   'team1_bowler_economy_last_5', 
                                   'team2_bowler_economy_last_5', 
                                   'team1_wickets_last_5', 
                                   'team2_wickets_last_5', 
                                   'team1_boundaries_last_5', 
                                   'team2_boundaries_last_5', 
                                   'toss_win_impact', 
                                   'team1_experience', 
                                   'team2_experience']]

# Save the DataFrame to a CSV file
additional_features_df.to_csv('./additional_features.csv', index=False)

# Create a DataFrame with feature descriptions
feature_descriptions = {
    'feature': [
        'team1_avg_runs_last_5', 
        'team2_avg_runs_last_5', 
        'team1_bowler_economy_last_5', 
        'team2_bowler_economy_last_5', 
        'team1_wickets_last_5', 
        'team2_wickets_last_5', 
        'team1_boundaries_last_5', 
        'team2_boundaries_last_5', 
        'toss_win_impact',
        'team1_experience', 
        'team2_experience'
    ],
    'description': [
        'Team1\'s average runs in the last 5 matches', 
        'Team2\'s average runs in the last 5 matches', 
        'Average economy rate of team1 bowlers in the last 5 matches', 
        'Average economy rate of team2 bowlers in the last 5 matches', 
        'Total wickets taken by team1 bowlers in the last 5 matches', 
        'Total wickets taken by team2 bowlers in the last 5 matches', 
        'Total boundaries (fours and sixes) scored by team1 in the last 5 matches', 
        'Total boundaries (fours and sixes) scored by team2 in the last 5 matches', 
        'Binary feature indicating if the toss winner is also the match winner', 
        'Average number of matches played by the team1 roster', 
        'Average number of matches played by the team2 roster'
    ]
}

feature_descriptions_df = pd.DataFrame(feature_descriptions)

# Save the DataFrame to a CSV file
feature_descriptions_df.to_csv('./Add_feature_descriptions.csv', index=False)


In [16]:
print(result_df.shape)

(1689, 23)


In [17]:
train_path = './train_data_with_samplefeatures.csv'
train_data = pd.read_csv(train_path)

print(train_data.shape)

(948, 23)


Merging with traindata(given)

In [18]:
merged_df = pd.concat([result_df, train_data]).reset_index(drop=True)

In [19]:
print(merged_df.shape)

(2637, 23)


In [20]:
merged_df.to_csv('./MergedTrainData.csv',index=False)